# Code

In [1]:
%pip install -q "nbformat==5.10.3"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q "ipywidgets"
import ipywidgets as widgets
%pip install -q "plotly"
import plotly.graph_objects as go

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
from IPython.display import display
from scipy.optimize import fsolve

In [4]:
from skimage import measure
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")

In [5]:
def renormalize(n, range1, range2):
    delta1 = range1[1] - range1[0]
    delta2 = range2[1] - range2[0]
    return (delta2 * (n - range1[0]) / delta1) + range2[0]

In [6]:
def intensity_func(x,y,z):
    return (x**2+y**2+z**2)

In [7]:
def determinant(kx,ky,kz,narr):

    if len(set(narr)) == 1:
        return kx**2+ky**2+kz**2-narr[0]**2

    else:
        n1,n2,n3 = narr
        
        return kx**4*n1**2+(ky**2+kz**2-n1**2)*(ky**2*n2**2+(kz**2-n2**2)*n3**2) + kx**2*(ky**2*(n1**2+n2**2)+kz**2*(n1**2+n3**2)-n1**2*(n2**2+n3**2))

In [8]:
def add_OA(narr,fig):

    n1,n2,n3 = narr
    stretcher = 1.5

    #on yz plane
    y0 = n3*np.sqrt((n1**2-n2**2)/(n3**2 - n2**2))
    z0 = np.sqrt(n1**2 - y0**2)
    fig.add_scatter3d(x=[0,0], y=[-stretcher*y0,stretcher*y0], z=[stretcher*z0,-stretcher*z0], 
                      mode='lines', line_width=6, line_color='red', showlegend=False, name='Asse Ottico')
    fig.add_scatter3d(x=[0,0], y=[-stretcher*y0,stretcher*y0], z=[-stretcher*z0,stretcher*z0], 
                      mode='lines', line_width=6, line_color='red', showlegend=False, name='Asse Ottico')
    
    #on xz plane
    x0 = n3*np.sqrt((-n1**2 + n2**2)/(-n1**2 + n3**2))
    z0 = np.sqrt(n2**2 - x0**2)    
    fig.add_scatter3d(x=[-stretcher*x0,stretcher*x0], y=[0,0], z=[stretcher*z0,-stretcher*z0], 
                      mode='lines', line_width=6, line_color='black', showlegend=False, name='Asse Ottico')
    fig.add_scatter3d(x=[-stretcher*x0,stretcher*x0], y=[0,0], z=[-stretcher*z0,stretcher*z0], 
                      mode='lines', line_width=6, line_color='black', showlegend=False, name='Asse Ottico')

    #on yz 
    x0 = n2*np.sqrt((n1**2 - n3**2)/(n1**2 - n2**2))
    y0 = np.sqrt(n3**2 - x0**2)    
    fig.add_scatter3d(x=[-stretcher*x0,stretcher*x0], y=[-stretcher*y0,stretcher*y0], z=[0,0],
                      mode='lines', line_width=6, line_color='blue', showlegend=False, name='Asse Ottico')
    fig.add_scatter3d(x=[-stretcher*x0,stretcher*x0], y=[stretcher*y0,-stretcher*y0], z=[0,0],
                      mode='lines', line_width=6, line_color='blue', showlegend=False, name='Asse Ottico')

    

In [9]:
def update_plot(n1, n2, n3):

    narr = np.array([n1,n2,n3])
    
    vmax = narr.max()+0.1
    xmin,xmax = -vmax,vmax
    ymin,ymax = -vmax,vmax
    zmin,zmax = -vmax,vmax
    X, Y, Z = np.mgrid[xmin:xmax:60j, -vmax:vmax:60j, -vmax:vmax:60j]
    F = determinant(X,Y,Z,narr)

    mins = np.array([xmin, -vmax, -vmax])
    maxes = np.array([xmax, vmax, vmax])
    ranges = maxes - mins

    verts, faces = measure.marching_cubes(F, 0.035)[:2]
    verts_scaled = verts * ranges / np.array(F.shape) + mins

    x, y, z = verts_scaled.T
    I, J, K = faces.T
    intensity = intensity_func(x,y,z)
    mesh = go.Mesh3d(x=x,
                         y=y,
                         z=z,
                         colorscale='viridis', 
                         intensity= intensity,
                         opacity = 0.6,
                         i=I,
                         j=J,
                         k=K,
                         name='',
                         lighting=dict(specular=1)
                        )  

    fig = go.Figure(mesh)
    fig.update_traces(showscale=False)
    add_OA(narr,fig)
    fig.update_layout(scene_aspectmode='cube',width=600, height=600, 
                      scene = dict(xaxis = dict(ticks="",showticklabels=False,title='kx',range=[-xmax,xmax]),
                                   yaxis = dict(showticklabels=False,title='ky',range=[-ymax,ymax]),
                                   zaxis = dict(showticklabels=False,title='kz',range=[-zmax,zmax]),),
                      font_size=11)
    fig.show()

In [10]:
initial_n1 = 1.
initial_n2 = 1.
initial_n3 = 1.

# Create sliders for adjusting slopes
n1_slider = widgets.FloatSlider(value=initial_n1, min=1, max=3, step=0.02, description='n1 :')
n2_slider = widgets.FloatSlider(value=initial_n2, min=1, max=3, step=0.02, description='n2 :')
n3_slider = widgets.FloatSlider(value=initial_n3, min=1, max=3, step=0.02, description='n3 :')

In [11]:
interactive_plot = widgets.interactive(update_plot, n1=n1_slider, n2=n2_slider, n3=n3_slider)

# Superficie normale

### Descrizione

Superficie normale di un mezzo dielettrico.</br>
Variando gli indici di rifrazione principali $n_1$, $n_2$, $n_3$, si può avere un mezzo dielettrico isotropo, anisotropo uniassiale e anisotropo biassiale.</br>
Nei casi uniassiale e biassiale vengono anche indicati gli assi ottici.


### Utilizzo
Per eseguire il notebook, dal menu in alto: `Run > Run All Cells`

In [ ]:
display(interactive_plot)